In [14]:
import numpy as np
from qiskit import Aer
from qiskit.providers.aer import StatevectorSimulator, AerSimulator
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
    HDF5Driver,
    FCIDumpDriver
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper,BravyiKitaevMapper, BravyiKitaevSuperFastMapper, ParityMapper
# BravyiKitaevSuperFastMapper: https://arxiv.org/pdf/1712.00446.pdf

## The Hartree-Fock initial state

In [15]:
molecule = Molecule(
    geometry=[["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 0.9]]], charge=0, multiplicity=1
)
driver = ElectronicStructureMoleculeDriver(
    molecule, basis="ano", driver_type=ElectronicStructureDriverType.PYSCF
)

## The mapping from fermions to qubits¶

In [16]:
es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()
print(second_q_op[0])

In [13]:
qubit_converter = QubitConverter(JordanWignerMapper(), two_qubit_reduction=True, z2symmetry_reduction='auto')
qubit_op = qubit_converter.convert(second_q_op[0], num_particles=es_problem.num_particles)
print("Is operator Hermitian?", qubit_op.is_hermitian())
print(qubit_op)

Is operator Hermitian? True
1.517253575675059 * IIIIIIII
- 0.9487759991891249 * ZIIIIIII
- 0.7310125307062609 * IZIIIIII
+ 0.09604445496183861 * ZZIIIIII
- 0.3931349636502246 * IIZIIIII
+ 0.09239202619508673 * ZIZIIIII
+ 0.08962115356493582 * IZZIIIII
- 0.27377531095433766 * IIIZIIII
+ 0.12061617973424082 * ZIIZIIII
+ 0.10359050708576532 * IZIZIIII
+ 0.08536008686676534 * IIZZIIII
- 0.9487759991891248 * IIIIZIII
+ 0.1678396290437839 * ZIIIZIII
+ 0.13493140125007025 * IZIIZIII
+ 0.11170313195448163 * IIZIZIII
+ 0.15083856072943122 * IIIZZIII
- 0.7310125307062607 * IIIIIZII
+ 0.13493140125007025 * ZIIIIZII
+ 0.1171300918523781 * IZIIIZII
+ 0.09829136508468025 * IIZIIZII
+ 0.12868106775523064 * IIIZIZII
+ 0.09604445496183861 * IIIIZZII
- 0.3931349636502247 * IIIIIIZI
+ 0.11170313195448163 * ZIIIIIZI
+ 0.09829136508468025 * IZIIIIZI
+ 0.09866475045343676 * IIZIIIZI
+ 0.10960417197892046 * IIIZIIZI
+ 0.09239202619508673 * IIIIZIZI
+ 0.08962115356493582 * IIIIIZZI
- 0.27377531095433744 * III

## To unitaries

In [4]:
# from qiskit.opflow import Suzuki
# unitary_converter = Suzuki()
# composed_op = unitary_converter.convert(qubit_op)

emiH = qubit_op.exp_i().to_matrix() #exp(-iH)
num_qubits_required = int(np.log2(emiH.shape[0]))
print("Number of Qubits Needed:", num_qubits_required)
emiH.shape

NameError: name 'qubit_op' is not defined

In [8]:
from qiskit import QuantumCircuit, transpile

circ = QuantumCircuit(num_qubits_required)
circ.unitary(emiH, list(range(num_qubits_required)))
trans_circ = transpile(circ, basis_gates=['cx', 'id', 'rz', 'sx', 'x'], optimization_level=1)
trans_circ.measure_all()

In [9]:
print("depth:", trans_circ.depth())
print("width:", trans_circ.width())
print("Operations:", trans_circ.count_ops())

depth: 467
width: 8
Operations: OrderedDict([('rz', 252), ('sx', 175), ('cx', 112), ('measure', 4), ('x', 1), ('barrier', 1)])
